This notebook is part of the `kikuchipy` documentation https://kikuchipy.org.
Links to the documentation won't work from the notebook.

# Visualizing patterns

The [EBSD](../reference.rst#kikuchipy.signals.EBSD) and
[EBSDMasterPattern](../reference.rst#kikuchipy.signals.EBSDMasterPattern) signals
have a powerful and versatile
[plot()](http://hyperspy.org/hyperspy-doc/current/api/hyperspy.signal.html#hyperspy.signal.BaseSignal.plot)
method provided by HyperSpy. Its uses are greatly detailed in HyperSpy's
[visualisation user guide](http://hyperspy.org/hyperspy-doc/current/user_guide/visualisation.html).
This section details example uses specific to EBSD and EBSDMasterPattern
signals.

Let's import the necessary libraries and a Nickel EBSD test data set
<cite data-cite="aanes2019electron">Ånes et al. (2019)</cite>:

In [ ]:
# Exchange inline for notebook or qt5 (from pyqt) for interactive plotting
%matplotlib inline

import hyperspy.api as hs
import kikuchipy as kp
import matplotlib.pyplot as plt
import numpy as np
from orix import io, plot, quaternion, vector
import pyvista
import skimage.exposure as ske
import skimage.transform as skt


pyvista.global_theme.window_size = [700, 700]
pyvista.set_jupyter_backend("pythreejs")

In [ ]:
# Use kp.load("data.h5") to load your own data
s = kp.data.nickel_ebsd_large(allow_download=True)  # External download
s

## Navigate in custom map

Correlating results from e.g. crystal and phase structure determination, i.e.
indexing, with experimental patterns can inform their interpretation. When
calling `plot()` without any input parameters, the navigator map is a grey scale
image with pixel values corresponding to the sum of all detector intensities
within that pattern:

In [ ]:
s.plot()

The upper panel shows the navigation axes, in this case 2D, with the current
beam position in the upper left corner shown as a red square the size of one
pixel. This square can be made larger/smaller with <kbd>+</kbd>/<kbd>-</kbd>.
The square can be moved either by the keyboard arrows or the mouse. The lower
panel shows the image on the detector in the current beam position.

Any
[BaseSignal](http://hyperspy.org/hyperspy-doc/current/api/hyperspy.signal.html#hyperspy.signal.BaseSignal)
signal with a 2D `signal_shape` corresponding to the scan
`navigation_shape` can be passed in to the `navgiator` parameter in
[plot()](http://hyperspy.org/hyperspy-doc/current/api/hyperspy.signal.html#hyperspy.signal.BaseSignal.plot),
including a virtual image showing diffraction contrast, any quality metric map,
or an orientation map or a phase map.

## Virtual image

A virtual backscatter electron (VBSE) image created from any detector region of
interest with the
[get_virtual_bse_intensity()](../reference.rst#kikuchipy.signals.EBSD.get_virtual_bse_intensity)
method or
[get_rgb_image()](../reference.rst#kikuchipy.generators.VirtualBSEGenerator.get_rgb_image)
explained in the
[virtual backscatter electron imaging](virtual_backscatter_electron_imaging.rst)
section, can be used as a navigator for a scan `s`:

In [ ]:
vbse_gen = kp.generators.VirtualBSEGenerator(s)
print(vbse_gen)
print(vbse_gen.grid_shape)

In [ ]:
vbse_rgb = vbse_gen.get_rgb_image(r=(3, 1), b=(3, 2), g=(3, 3))
vbse_rgb

In [ ]:
s.plot(navigator=vbse_rgb, cmap="viridis")

## Any image

Images made into a
[Signal2D](http://hyperspy.org/hyperspy-doc/current/api/hyperspy._signals.signal2d.html#hyperspy._signals.signal2d.Signal2D)
can be used as navigators, like a quality metric map like the
[image quality map](feature_maps.ipynb#Image-quality) calculated using
[get_image_quality()](../reference.rst#kikuchipy.signals.EBSD.get_image_quality):

In [ ]:
s.remove_static_background()
s.remove_dynamic_background()

In [ ]:
iq = s.get_image_quality()
s_iq = hs.signals.Signal2D(iq)
s.plot(navigator=s_iq, scalebar=False)

We can obtain an RGB signal from an RGB image using
[get_rgb_navigator()](../reference.rst#kikuchipy.draw.get_rgb_navigator).
Let's load an orientation map from dictionary indexing in the
[pattern matching](pattern_matching.rst) user guide

In [ ]:
rgb_z = plt.imread("../_static/image/visualizing_patterns/ni_large_rgb_z.png")
rgb_z = rgb_z[..., :3]  # Drop the alpha channel
s_rgb_z = kp.draw.get_rgb_navigator(rgb_z)

s.plot(navigator=s_rgb_z, colorbar=False)

## Plot multiple signals

HyperSpy provides the function
[plot_signals()](http://hyperspy.org/hyperspy-doc/current/api/hyperspy.drawing.utils.html#hyperspy.drawing.utils.plot_signals)
to plot multiple signals with the same navigator, as explained in their
[user guide](http://hyperspy.org/hyperspy-doc/current/user_guide/visualisation.html#plotting-several-signals).
This enables e.g. plotting of experimental and best matching simulated patterns
side-by-side as a visual inspection of indexing results. See the
[pattern matching user guide](pattern_matching.ipynb#Analyze-indexing-results)
for a demonstration.

## Plot master patterns

[EBSDMasterPattern](../reference.rst#kikuchipy.signals.EBSDMasterPattern) signals
can be navigated along their energy axis and/or their northern/southern
hemisphere. Let's reload the Nickel master pattern used in the previous section,
but this time in the stereographic projection.

In [ ]:
# Only a single energy, 20 keV
mp_stereo = kp.data.nickel_ebsd_master_pattern_small(
    projection="stereographic", hemisphere="both"
)
print(mp_stereo.axes_manager)

As can be seen from the axes manager, the master pattern has two navigation
axes, the northern and southern hemispheres, thus, when plotting, we get a
navigation slider

In [ ]:
mp_stereo.plot()

We can plot the master pattern on the sphere, provided it is in the
stereographic projection and that both hemispheres are loaded if it is
non-centrosymmetric, using
[EBSDMasterPattern.plot_spherical()](../reference.rst#kikuchipy.signals.EBSDMasterPattern.plot_spherical).
The initial orientation of the sphere corresponds to the orientation of the
stereographic and Lambert projections.

In [ ]:
mp_stereo.plot_spherical(style="points")

This plot uses the [pythreejs](https://github.com/jupyter-widgets/pythreejs)
package for plotting in the notebook, which must be installed separately. If we
want to plot the master pattern in a separate window using
[pyvista](https://docs.pyvista.org) instead, we can pass
`plotter_kwargs={"notebook": False}"` to `plot_spherical()`.